In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StopWordsRemover
from textblob import TextBlob

import requests
import json

# import nltk
# nltk.download("stopwords")
# stopwordList = nltk.corpus.stopwords.words('portuguese')

In [3]:
if __name__ == "__main__":
    # create Spark session
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
    # read the tweet data from socket
    lines = spark.readStream.format("socket").option("host", "127.0.0.1").option("port", 9999).load()
#     remover = StopWordsRemover(inputCol='word', outputCol="stopWordsRem", stopWords=stopwordList)
    
    words = lines.select(
        explode(
            split(lower(lines.value), " ")
        ).alias('word')
    )
    words = words.filter((words.word == 'buy')  | 
                         (words.word == 'sell') |
                         (words.word == 'bull') |
                         (words.word == 'bear') |
                         (words.word == 'moon') |
                         (words.word == 'crash'))
    
#     words = words.filter(words['word'] == '')
#     words = lines.select(
#         explode(
#             split(
#                 remover.transform(lower(lines.value).alias('word')).select("stopWordsRem").alias('word'),
#                 " ")
#         ).alias("word")
#     )
    wordCounts = words.groupBy("word").count()
    
    query =  wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("memory") \
    .queryName("streaming_data") \
    .trigger(processingTime='3 seconds') \
    .start()

In [59]:
df = spark.sql("SELECT * FROM streaming_data")

In [82]:
df.filter(df['word'] == i).select('count').head()[0]

27

In [95]:
inputs = [0, 0, 0]
input_list = ['buy', 'bull', 'moon', 'sell', 'bear', 'crash']

for i in input_list[0:3]:
    try:
        inputs[0] += df.filter(df['word'] == i).select('count').head()[0]
    
    except Exception:
        pass

for i in input_list[3:6]:
    try:
        inputs[1] += df.filter(df['word'] == i).select('count').head()[0]
    
    except Exception:
        pass

inputs[2] = inputs[0] - inputs[1]

try:
    inputs -= last_state
    
except Exception:
    pass
    
last_state = inputs

In [96]:
inputs

[41, 1, 40]

In [25]:
import time
import pandas as pd
import seaborn as sns

for i in range(10):
    print(len(spark.sql("SELECT * FROM streaming_data").toPandas()))
#     sns.barplot(x = 'word', y = 'count', data = pandas_df.iloc[0:5,:]);
    time.sleep(3)

821
821
821
821
821
821
821
821
821
821


In [6]:
query.stop()

In [81]:
df = spark.sql("SELECT * FROM streaming_data").orderBy('count', ascending = False)
df.show()

+--------------------+-----+
|                word|count|
+--------------------+-----+
|            euphoria|   13|
|                  in|    5|
|                  is|    4|
|                    |    4|
|           #euphoria|    4|
|                  de|    4|
|                   o|    4|
|                 que|    4|
|                  ta|    3|
|              parece|    3|
|               maddy|    3|
|                foto|    3|
|          euphoriart|    3|
|                essa|    3|
|         @pwrsphone:|    3|
|https://t.co/dsrk...|    3|
|                 s2,|    3|
|                 ep4|    3|
|                  em|    3|
|             douglas|    3|
+--------------------+-----+
only showing top 20 rows



In [98]:
import torch
import random
import numpy as np
from collections import deque
from scipy.stats import bernoulli
from model import Linear_QNet, QTrainer
from dogAI import DogGameAI
from helper import plot

ModuleNotFoundError: No module named 'model'

In [ ]:
import torch
import random
import numpy as np
from collections import deque
from scipy.stats import bernoulli
from model import Linear_QNet, QTrainer
from dogAI import DogGameAI
from helper import plot

max_memory = 100_000
batch_size = 1000
learning_rate = 0.005

class Agent:

	def __init__(self):
		self.n_games = 0
		self.epsilon = 0 # aleatoriedade
		self.gamma = 0 # taxa de desconto
		self.memory = deque(maxlen = max_memory) # se alcança o limite: popleft()
		self.model = Linear_QNet(10, 256, 5)
		self.trainer = QTrainer(self.model, lr=learning_rate, gamma=self.gamma)

	def get_state(self, game):
		state = [
			game.player.x, game.player.y, game.player.mana, game.score, # Player
			game.biscoito.x, game.biscoito.y, # Biscoito
			game.monster_1.x, game.monster_1.y, game.monster_2.x, game.monster_2.y # Mobs
		]

		return state

	def remember(self, state, action, reward, next_state, game_over):
		self.memory.append((state, action, reward, next_state, game_over))

	def train_long_memory(self):
		if len(self.memory) > batch_size:
			mini_sample = random.sample(self.memory, batch_size) # list of tuples

		else:
			mini_sample = self.memory

		states, actions, rewards, next_states, game_overs = zip(*mini_sample)
		self.trainer.train_step(states, actions, rewards, next_states, game_overs)

	def train_short_memory(self, state, action, reward, next_state, game_over):
		self.trainer.train_step(state, action, reward, next_state, game_over)

	def get_action(self, state):
		# random moves: tradeoff exploration / exploitation
		# self.epsilon = 620 - 100*np.log(self.n_games)
		self.epsilon = 150 - self.n_games
		final_move = [0, 0, 0, 0, 0, 0]
		if random.randint(0, 200) < self.epsilon:
			final_move = torch.tensor(bernoulli.rvs(size = 5, p = 0.5), dtype = int)

		else:
			state0 =  torch.tensor(state, dtype=torch.float)
			final_move = self.model(state0)
			print(final_move)

		# print(type(final_move))

		return final_move

def train():
	plot_scores = []
	plot_mean_scores = []
	total_score = 0
	best_score = 0
	agent = Agent()
	game = DogGameAI()
	while True:
		# get old state
		state_old = agent.get_state(game)

		# get move
		final_move = agent.get_action(state_old)

		# perform move and get new state
		reward, done, score = game.play_step(final_move)
		state_new = agent.get_state(game)

		# train short memory
		agent.train_short_memory(state_old, final_move, reward, state_new, game.game_over)

		# remember
		agent.remember(state_old, final_move, reward, state_new, game.game_over)

		if game.game_over:
			# train long memory / replay memory / experience replay
			game.reset()
			agent.n_games += 1
			agent.train_long_memory()

			if score > best_score:
				best_score = score
				agent.model.save()

			print(f"Game:{agent.n_games} Score {score} Best Score {best_score} ")

			plot_scores.append(score)
			total_score += score
			mean_score = total_score / agent.n_games
			plot_mean_scores.append(mean_score)
			plot(plot_scores, plot_mean_scores)

if __name__ == '__main__':
	train()

In [18]:
url = "https://api.livecoinwatch.com/coins/single"

payload = json.dumps({
  "currency": "USD",
  "code": "SHIB",
  "meta": False
})
headers = {
  'content-type': 'application/json',
  'x-api-key': 'be34977f-9765-404f-ab83-a97678e83da2'
}

response = requests.request("POST", url, headers=headers, data=payload)

response.text

'{"rate":0.00003106824934034432,"volume":4228292276,"cap":18319975045}'

In [38]:
float(response.text.split(',')[0].split(':')[1])

3.106824934034432e-05